In [1]:
!pip install virtualenv
!virtualenv ai_course_env

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 212.3 MB/s eta 0:00:00
created virtual environment CPython3.10.14.final.0-64 in 7764ms
  creator CPython3Posix(dest=/home/ahmeddoha/PycharmProjects/ai_course/class_04/ai_course_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/home/ahmeddoha/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.1.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!pip install accelerate==0.33.0
!pip install apify-client==1.6.4
!pip install arxiv==2.1.0
!pip install duckduckgo_search==5.2.1
!pip install faiss-cpu==1.8.0
!pip install google-api-python-client==2.124.0
!pip install google-cloud-aiplatform==1.60.0
!pip install google-generativeai==0.4.1
!pip install gpt4all==2.3.2
!pip install gradio==3.50.2
!pip install huggingface-hub==0.24.5
!pip install jupyter
!pip install lanarky==0.8.6
!pip install langchain-anthropic==0.1.4
!pip install langchain-chroma==0.1.2
!pip install langchain-cli==0.0.21
!pip install langchain-cohere==0.1.1
!pip install langchain-core==0.1.42
!pip install langchain-experimental==0.0.55
!pip install langchain-google-genai==1.0.1
!pip install langchain-google-vertexai==0.1.2
!pip install langchain-openai==0.1.1
!pip install langchain[docarray]==0.1.13
!pip install langchainhub==0.1.20
!pip install langserve[all]==0.0.51
!pip install numexpr==2.10.0
!pip install opencv-python==4.9.0.80
!pip install pandoc==1.1.0  # needs system install as well!
!pip install pdf2image==1.17.0
!pip install pdfminer.six==20231228
!pip install pikepdf==8.15.1
!pip install pillow_heif==0.16.0
!pip install pymupdf==1.24.1
!pip install pypdf==4.1.0
!pip install replicate==0.30.1
!pip install ruff==0.3.4
!pip install sentence-transformers==2.6.1
!pip install streamlit==1.37.0
!pip install tiktoken==0.6.0
!pip install unstructured==0.15.0
!pip install wikipedia==1.4.0
!pip install wolframalpha==5.0.0

In [ ]:
import os
OPENAI_API_KEY = "OPENAI_API_KEY"
# I'm omitting all other keys
def set_environment():
    variable_dict = globals().items()
    for key, value in variable_dict:
        if "API" in key or "ID" in key:
            os.environ[key] = value

In [ ]:
"""Utility functions and constants.

I am having some problems caching the memory and the retrieval. When
I decorate for caching, I get streamlit init errors.
"""

import logging
import pathlib
from typing import Any

from langchain.memory import ConversationBufferMemory
from langchain_community.document_loaders.epub import UnstructuredEPubLoader
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders.word_document import UnstructuredWordDocumentLoader
from langchain_core.documents import Document
from streamlit.logger import get_logger

logging.basicConfig(encoding="utf-8", level=logging.INFO)
LOGGER = get_logger(__name__)


def init_memory():
    """Initialize the memory for contextual conversation.

    We are caching this, so it won't be deleted
     every time, we restart the server.
    """
    return ConversationBufferMemory(
        memory_key="chat_history", return_messages=True, output_key="answer"
    )


LOGGER.info("init memory")
MEMORY = init_memory()


class EpubReader(UnstructuredEPubLoader):
    def __init__(self, file_path: str | list[str], **unstructured_kwargs: Any):
        super().__init__(file_path, **unstructured_kwargs, mode="elements", strategy="fast")


class DocumentLoaderException(Exception):
    pass


class DocumentLoader(object):
    """Loads in a document with a supported extension."""

    supported_extensions = {
        ".pdf": PyPDFLoader,
        ".txt": TextLoader,
        ".epub": EpubReader,
        ".docx": UnstructuredWordDocumentLoader,
        ".doc": UnstructuredWordDocumentLoader,
    }


def load_document(temp_filepath: str) -> list[Document]:
    """Load a file and return it as a list of documents.

    Doesn't handle a lot of errors at the moment.
    """
    ext = pathlib.Path(temp_filepath).suffix
    loader = DocumentLoader.supported_extensions.get(ext)
    if not loader:
        raise DocumentLoaderException(
            f"Invalid extension type {ext}, cannot load this type of file"
        )

    loaded = loader(temp_filepath)
    docs = loaded.load()
    logging.info(docs)
    return docs


In [ ]:
"""Chat with retrieval and embeddings."""

import os
import tempfile

# from class04_llm_rag_auffarth.chapter5.config import set_environment
from langchain.chains.base import Chain
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.chains.flare.base import FlareChain
from langchain.chains.moderation import OpenAIModerationChain
from langchain.chains.sequential import SequentialChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_community.vectorstores.docarray import DocArrayInMemorySearch
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# from class04_llm_rag_auffarth.chapter5.chat_with_retrieval.utils import LOGGER, MEMORY, load_document

set_environment()

LOGGER.info("setup LLM")
# Setup LLM and QA chain; set temperature low to keep hallucinations in check
LLM = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, streaming=True)


LOGGER.info("configure_retriever")


def configure_retriever(docs: list[Document], use_compression: bool = False) -> BaseRetriever:
    """Retriever to use."""
    # Split each document documents:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)

    # Create embeddings and store in vectordb:
    # embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    # Create vectordb with single call to embedding model for texts:
    vectordb = DocArrayInMemorySearch.from_documents(splits, embeddings)
    retriever = vectordb.as_retriever(
        search_type="mmr",
        search_kwargs={"k": 5, "fetch_k": 7, "include_metadata": True},
    )
    if not use_compression:
        return retriever

    embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.2)
    return ContextualCompressionRetriever(
        base_compressor=embeddings_filter,
        base_retriever=retriever,
    )


def configure_chain(retriever: BaseRetriever, use_flare: bool = True) -> Chain:
    """Configure chain with a retriever.

    Passing in a max_tokens_limit amount automatically
    truncates the tokens when prompting your llm!
    """
    output_key = "response" if use_flare else "answer"
    MEMORY.output_key = output_key
    params = dict(
        llm=LLM,
        retriever=retriever,
        memory=MEMORY,
        verbose=True,
        max_tokens_limit=4000,
    )
    if use_flare:
        # different set of parameters and init
        return FlareChain.from_llm(**params)
    return ConversationalRetrievalChain.from_llm(**params)


def configure_retrieval_chain(
    uploaded_files,
    use_compression: bool = False,
    use_flare: bool = False,
    use_moderation: bool = False,
) -> Chain:
    """Read documents, configure retriever, and the chain."""
    docs = []
    temp_dir = tempfile.TemporaryDirectory()
    for file in uploaded_files:
        temp_filepath = os.path.join(temp_dir.name, file.name)
        with open(temp_filepath, "wb") as f:
            f.write(file.getvalue())
        docs.extend(load_document(temp_filepath))

    retriever = configure_retriever(docs=docs, use_compression=use_compression)
    chain = configure_chain(retriever=retriever, use_flare=use_flare)
    if not use_moderation:
        return chain

    input_variables = ["user_input"] if use_flare else ["chat_history", "question"]
    moderation_input = "response" if use_flare else "answer"
    moderation_chain = OpenAIModerationChain(input_key=moderation_input)
    return SequentialChain(chains=[chain, moderation_chain], input_variables=input_variables)


In [ ]:
!pip install --upgrade docarray

In [ ]:
"""Document loading functionality.

Run like this:

PYTHONPATH=. streamlit run /class_04/chat_with_retrieval/app.py

"""

import streamlit as st
from streamlit.external.langchain import StreamlitCallbackHandler

# from class_04.chat_with_retrieval.chat_with_documents import configure_retrieval_chain
# from class_04.chat_with_retrieval.utils import LOGGER, MEMORY, DocumentLoader

LOGGER.info("Show title")
st.set_page_config(page_title="LangChain: Chat with Documents", page_icon="🦜")
st.title("🦜 LangChain: Chat with Documents")


LOGGER.info("Upload files")
uploaded_files = st.sidebar.file_uploader(
    label="Upload files",
    type=list(DocumentLoader.supported_extensions.keys()),
    accept_multiple_files=True,
)
if not uploaded_files:
    st.info("Please upload documents to continue.")
    st.stop()

# use compression by default:
use_compression = st.checkbox("compression", value=False)
use_flare = st.checkbox("flare", value=False)
use_moderation = st.checkbox("moderation", value=False)

LOGGER.info("Configure chain")
CONV_CHAIN = configure_retrieval_chain(
    uploaded_files,
    use_compression=use_compression,
    use_flare=use_flare,
    use_moderation=use_moderation,
)

LOGGER.info("Clear button")
if st.sidebar.button("Clear message history"):
    MEMORY.chat_memory.clear()

avatars = {"human": "user", "ai": "assistant"}

if len(MEMORY.chat_memory.messages) == 0:
    st.chat_message("assistant").markdown("Ask me anything!")

for msg in MEMORY.chat_memory.messages:
    st.chat_message(avatars[msg.type]).write(msg.content)

LOGGER.info("Chat interface")
container = st.container()
assistant = st.chat_message("assistant")
if user_query := st.chat_input(placeholder="Give me 3 keywords for what you have right now"):
    st.chat_message("user").write(user_query)
    stream_handler = StreamlitCallbackHandler(container)
    with st.chat_message("assistant"):
        if use_flare:
            params = {"user_input": user_query}
        else:
            params = {
                "question": user_query,
                "chat_history": MEMORY.chat_memory.messages,
            }
        response = CONV_CHAIN.invoke(
            {"question": user_query, "chat_history": MEMORY.chat_memory.messages},
            callbacks=[stream_handler],
        )
        # Display the response from the chatbot
        if response:
            container.markdown(response)
